In [ ]:
!pip install openai
!pip install pytorch-lightning
!pip install transformers
!pip install  gradio

In [ ]:
## chatGPT with Gradio 起手式
## 在你的資料夾新增 .env 檔案，並在裡面寫入 API_KEY=你的API金鑰
import openai
import gradio as gr


## AI 建議
def get_advice(bmi,temp, API_KEY, model="gpt-3.5-turbo"):
    openai.api_key = API_KEY
    messages = [{"role": "system", "content": "You can provide some dietary advice based on \
                 the user's BMI value. You can only give up to 3 suggestions"},
                {"role": "user", "content": f'My BMI is {bmi}. What can I do to be healthier?'},]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

## 健身計畫
def get_gym(bmi,slide, temp, API_KEY, model="gpt-3.5-turbo"):
    openai.api_key = API_KEY
    messages = [{"role": "system", "content": "You are a great fitness coach and \
                 you will give users great fitness plans."},
                {"role": "user", "content": f'My BMI is {bmi}. I want a {slide}-point weight\
                  loss plan, from 1 to 10. The higher the number, the faster the weight loss.'},]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

def BMI(height, weight) -> int:
    height = int(height) / 100
    bmi = int(weight) / (height * height)
    if bmi < 18.5:
        return str(bmi)[:5], "過輕"
    elif bmi < 24:
        return str(bmi)[:5], "正常"
    elif bmi < 27:
        return str(bmi)[:5], "過重"
    elif bmi < 30:
        return str(bmi)[:5], "輕度肥胖"
    elif bmi < 35:
        return str(bmi)[:5], "中度肥胖"
    else:
        return str(bmi)[:5], "重度肥胖"



# 建立 components
height = gr.Textbox(
    label="身高",
    info="輸入你的身高(cm)",
    placeholder="Type your height here...")

weight = gr.Textbox(
    label="體重",
    info="輸入你的體重(kg)",
    placeholder="Type your weight here...",)

output_bmi = gr.Textbox(
    value="",
    label="BMI 值",
    info="顯示BMI 數字",
    placeholder="BMI")

output_state = gr.Textbox(
    value="",
    label="BMI 結果",
    info="診斷",
    placeholder="顯示診斷結果")

advice = gr.Textbox(
    label="AI Advice",
    info="請選擇以下按鈕讓AI 根據你的BMI值給予的建議",
    placeholder="Ouput Text here...",
    lines=5,)

btn = gr.Button(
    value="計算BMI值",
    variant="primary", scale=1)

btn_advice  = gr.Button(
    value="AI 建議",
    variant="primary", scale=2)

btn_gym = gr.Button(
    value="AI 健身計畫",
    variant="primary", scale=1)

key_box = gr.Textbox(
    label="Enter your API key",
    info="You have to provide your own OPENAI_API_KEY for this app to function properly",
    placeholder="Type OpenAI API KEY here...",
    type="password")


slider = gr.Slider(
    minimum=1,
    maximum=10,
    step=1,
    label="減重速度",
    value=5,
    info="請選擇你的減重速度，數字越大，減重越快",
    )

temperature = gr.Slider(
    minimum=0,
    maximum=1.0,
    value=0.3,
    step=0.05,
    label="Temperature",
    info=(
        "Temperature controls the degree of randomness in token selection. Lower "
        "temperatures are good for prompts that expect a true or correct response, "
        "while higher temperatures can lead to more diverse or unexpected results. "
    ),
)

with gr.Blocks() as demo:
    gr.Markdown("""
    # BMI 計算器
    簡易測量你的BMI值
    """)
    with gr.Column():
        with gr.Row():
            height.render()  # 顯示身高輸入框
            weight.render() # 顯示體重輸入框
        
        with gr.Row():
            output_bmi.render() # 顯示BMI值結果
            output_state.render() # 顯示BMI診斷結果

        btn.render() # 顯示計算BMI值按鈕
        btn.click(fn=BMI, 
                     inputs=[height, weight], 
                     outputs=[output_bmi,output_state])

        advice.render() # 顯示AI建議結果的文字框
        
        with gr.Row(): 
            key_box.render() # 顯示API金鑰輸入框
            btn_advice.render() # 顯示AI建議按鈕
            btn_advice.click(fn=get_advice, inputs=[output_bmi,temperature,key_box], outputs=advice)
            btn_gym.render() # 顯示AI健身計畫按鈕
            btn_gym.click(fn=get_gym, inputs=[output_bmi,slider,temperature, key_box], outputs=advice)
    
        with gr.Accordion("settings", open=True):
            slider.render()
            temperature.render()

demo.launch()



